In [1]:
import numpy as np
import argparse
import time
import cv2
import os
import pandas as pd
import fiftyone as fo
import fiftyone.zoo as foz
dataset = foz.load_zoo_dataset(
              "open-images-v6",
              split="train",
              label_types=["detections"],
              classes=["Bottle", "Tin can"],
              max_samples=100,
              seed = 51
          )

 100% |██████████████████| 100/100 [50.1s elapsed, 0s remaining, 5.5 images/s]      
Dataset info written to 'C:\Users\DELL\fiftyone\open-images-v6\info.json'
Loading 'open-images-v6' split 'train'
 100% |█████████████████| 100/100 [2.5s elapsed, 0s remaining, 40.2 samples/s]      
Dataset 'open-images-v6-train-100' created


In [6]:
import os
base_path="C:/Users/DELL/fiftyone/open-images-v6/validation"
images=os.path.sep.join([base_path,'data'])
annotations=os.path.sep.join([base_path,'labels/detection.csv'])

In [7]:
# Lets Load Dataset
# airplanes annotation is a Csv file thats why we can see through with rows

rows= open(annotations).read().strip().split("\n")

# lets make three list where we save our exact bounding boxes
data=[]
targets=[]
filenames=[]
# After load we have to split dataset according to images
# import some usefull libraries
import cv2
from tensorflow.keras.preprocessing.image import load_img
# we also save images into array format so import img_array library too
from tensorflow.keras.preprocessing.image import img_to_array
for row in rows:
  row=row.split(",")
  # we always create rectangle with h+w so we have to know where exactly we should start from
  (filename,startX,startY,endX,endY)=row
  print(row)  
  imagepaths=os.path.sep.join([images,filename])
  image=cv2.imread(imagepaths)
  (h,w)=image.shape[:2]

  # initializing starting point
  # Why we take in float because when we convert into array so then will trouble happen
  startX = float(startX) / w
  startY = float(startY) / h
  # Also initialize ending point 
  endX = float(endX) / w
  endY = float(endY) / h
  #load image and give them default size
  image=load_img(imagepaths,target_size=(224,224))
  # see here if we cant take it into float then we face trouble 
  image=img_to_array(image)

  # Lets append into data , targets ,filenames
  targets.append((startX,startY,endX,endY))
  filenames.append(filename)
  data.append(image)

['003232584a062b07', '0.1296875', '0.3375', '0.31875', '0.68958336']


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
# Normalizing Data here also we face would face issues if we take input as integer
import numpy as np
data=np.array(data,dtype='float32') / 255.0
targets=np.array(targets,dtype='float32')

# we should seperate data into train and split so import sklearn library 
from sklearn.model_selection import train_test_split

# split into testing and training
split=train_test_split(data,targets,filenames,test_size=0.10,random_state=42)

# lets split into steps
(train_images,test_images) = split[:2]
(train_targets,test_targets) = split[2:4]
(train_filenames,test_filenames) = split[4:]

# lets import pre trained VGG16 Which is already Builtin for computer vision
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input

# Imagenet is a competition every year held and VGG16 is winner of between  2013-14
# so here we just want limited layers so thats why we false included_top 
vgg=VGG16(weights='imagenet',include_top=False,input_tensor=Input(shape=(224,224,3)))

vgg.summary()

from tensorflow.keras.layers import Input,Flatten,Dense

# we use VGG16 as per our requirement not use whole 
vgg.trainable = False

flatten = vgg.output

flatten = Flatten()(flatten)

# Lets make bboxhead
bboxhead = Dense(128,activation="relu")(flatten)
bboxhead = Dense(64,activation="relu")(bboxhead)
bboxhead = Dense(32,activation="relu")(bboxhead)
bboxhead = Dense(4,activation="relu")(bboxhead)

# lets import Model
from tensorflow.keras.models import Model
model = Model(inputs = vgg.input,outputs = bboxhead)

model.summary()

# Lets fit our model 
# Optimization 
from tensorflow.keras.optimizers import Adam

opt = Adam(1e-4)

model.compile(loss='mse',optimizer=opt)

history = model.fit(train_images,train_targets,validation_data=(test_images,test_targets),batch_size=32,epochs=50,verbose=1)

# lets save model 
model.save('detect_Planes.h5')

from tensorflow.keras.models import load_model

model=load_model('/content/detect_Planes.h5')

imagepath='/content/drive/MyDrive/Applied_Ai_Course/Datasets/images/image_0111.jpg'

image = load_img(imagepath,
                 target_size=(224,224))
image = img_to_array(image) / 255.0
image = np.expand_dims(image,axis=0)

preds=model.predict(image)[0]
(startX,startY,endX,endY)=preds

import imutils

image=cv2.imread(imagepaths)
image=imutils.resize(image,width=600)

(h,w)=image.shape[:2]

startX=int(startX * w)
startY=int(startY * h)

endX=int(endX * w)
endY=int(endY * h)

cv2.rectangle(image,(startX,startY),(endX,endY),(0,255,0),3)

from google.colab.patches import cv2_imshow

import matplotlib.pyplot as plt
plt.imshow(image)
cv2.waitKey(0)